# Bqplot in depth

https://github.com/bloomberg/bqplot

## A Jupyter - d3.js bridge

bqplot is a jupyter interactive widget library bringing d3.js visualization to the Jupyter notebook.

- Apache Licensed

bqplot implements the abstractions of Wilkinson’s **The Grammar of Graphics** as interactive Jupyter widgets.

bqplot provides both
 - high-level plotting procedures with relevant defaults for common chart types,
 - lower-level descriptions of data visualizations meant for complex interactive visualization dashboards and applications involving mouse interactions and user-provided Python callbacks.

**Installation:**

```bash
conda install -c conda-forge bqplot
```

In [1]:
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import *
from traitlets import *

import numpy as np
import pandas as pd
import bqplot as bq
import datetime as dt

In [2]:
np.random.seed(0)
size = 100
y_data = np.cumsum(np.random.randn(size) * 100.0)
y_data_2 = np.cumsum(np.random.randn(size))
y_data_3 = np.cumsum(np.random.randn(size) * 100.)

x = np.linspace(0.0, 10.0, size)

price_data = pd.DataFrame(np.cumsum(np.random.randn(150, 2).dot([[0.5, 0.8], [0.8, 1.0]]), axis=0) + 100,
                          columns=['Security 1', 'Security 2'],
                          index=pd.date_range(start='01-01-2007', periods=150))

symbol = 'Security 1'
dates_all = price_data.index.values
final_prices = price_data[symbol].values.flatten()

# A simple plot with the pyplot API

In [3]:
from bqplot import pyplot as plt

In [4]:
plt.figure(1)
n = 100
plt.plot(np.linspace(0.0, 10.0, n), np.cumsum(np.random.randn(n)), 
         axes_options={'y': {'grid_lines': 'dashed'}})
plt.show()

### Scatter Plot

In [5]:
plt.figure(title='Scatter Plot with colors')
plt.scatter(y_data_2, y_data_3, color=y_data)
plt.show()

### Histogram

In [6]:
plt.figure()
plt.hist(y_data, colors=['OrangeRed'])
plt.show()

## We *do* allow pie charts

In [7]:
colors=['chocolate', 'orange', 'olive']

plt.figure()
plt.pie([0.45, 0.3, 0.25],
        labels=['Proportion of ' + color for color in colors],
        colors=colors, 
        label_color = 'white',
        font_size = '13px')
plt.show(display_toolbar=False)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=…

## Heatmap

In [8]:
x = np.linspace(-5, 5, 200)
y = np.linspace(-5, 5, 200)
X, Y = np.meshgrid(x, y)
color = np.cos(X**2 + Y**2)

In [9]:
fig = plt.figure(title='Cosine',
                 layout=Layout(width='600px', height='600px'),
                 min_aspect_ratio=1, max_aspect_ratio=1, padding_y=0)
plt.scales(scales={'color': bq.ColorScale(scheme='viridis', reverse=True)})
heatmap = plt.heatmap(color, x=x, y=y)
fig

Figure(axes=[ColorAxis(scale=ColorScale(reverse=True, scheme='viridis')), Axis(scale=LinearScale()), Axis(orie…

# Every component of the figure is an independent widget

## Updating values of a line chart

In [10]:
xs = bq.LinearScale()
ys = bq.LinearScale()
x = np.arange(100)
y = np.cumsum(np.random.randn(2, 100), axis=1) #two random walks

line = bq.Lines(x=x, y=y, scales={'x': xs, 'y': ys}, colors=['red', 'green'])
xax = bq.Axis(scale=xs, label='x', grid_lines='solid')
yax = bq.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='y', grid_lines='solid')

fig = bq.Figure(marks=[line], axes=[xax, yax], animation_duration=1000)
display(fig)

Figure(animation_duration=1000, axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='verti…

In [14]:
# update data of the line mark
line.y = np.cumsum(np.random.randn(2, 100), axis=1)

## Updating values of a Scatter

In [15]:
xs = bq.LinearScale()
ys = bq.LinearScale()
x, y = np.random.rand(2, 20)
scatt = bq.Scatter(x=x, y=y, scales={'x': xs, 'y': ys}, default_colors=['blue'])
xax = bq.Axis(scale=xs, label='x', grid_lines='solid')
yax = bq.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='y', grid_lines='solid')

fig = bq.Figure(marks=[scatt], axes=[xax, yax], animation_duration=1000)
display(fig)

Figure(animation_duration=1000, axes=[Axis(label='x', scale=LinearScale()), Axis(label='y', orientation='verti…

In [17]:
#data updates
scatt.x = np.random.rand(20) * 10
scatt.y = np.random.rand(20)

In [18]:
scatt.marker = 'diamond'
scatt.colors = ['red']

## The same holds for the attributes of scales, axes

In [19]:
xs.min = 4

In [20]:
xs.min = None

In [21]:
xax.label = 'Some label for the x axis'

## Scales at the center of the *Grammar of Graphics*

In [22]:
ys = bq.LinearScale()
xs1 = bq.LinearScale()
xs2 = bq.LinearScale()

x1, y1 = np.random.rand(2, 20)
x2, y2 = np.random.rand(2, 20)

x2 += 1000
y2 += 3

scatt1 = bq.Scatter(x=x1, y=y1, scales={'x': xs1, 'y': ys}, colors=['olive'])
scatt2 = bq.Scatter(x=x2, y=y2, scales={'x': xs2, 'y': ys}, colors=['orange'])

xax1 = bq.Axis(scale=xs1, label='x1', grid_lines='solid')
xax2 = bq.Axis(scale=xs2, label='x2', grid_lines='solid')

yax = bq.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='y', grid_lines='solid')

fig1 = bq.Figure(marks=[scatt1], axes=[xax1, yax], animation_duration=1000)
fig2 = bq.Figure(marks=[scatt2], axes=[xax2, yax], animation_duration=1000)

HBox([fig1, fig2])

In [23]:
scatt1.y = scatt1.y - 2

## Use bqplot figures as input widgets

In [24]:
xs = bq.LinearScale()
ys = bq.LinearScale()
x = np.arange(100)
y = np.cumsum(np.random.randn(100))

scatter = bq.Scatter(x=x, y=y, scales={'x': xs, 'y': ys})
xax = bq.Axis(scale=xs, label='x', grid_lines='solid')
yax = bq.Axis(scale=ys, orientation='vertical', tick_format='0.2f', label='y', grid_lines='solid')

## Selections

In [25]:
def interval_change_callback(change):
    db.value = str(change['new'])

intsel = bq.interacts.FastIntervalSelector(scale=xs, marks=[scatter])
intsel.observe(interval_change_callback, names=['selected'] )

db = widgets.Label()
db.value = str(intsel.selected)
display(db)

Label(value='None')

In [26]:
scatter.unselected_style={'opacity': 0.4}
scatter.selected_style={
     'fill': 'yellow'
}

In [27]:
fig = bq.Figure(marks=[scatter], axes=[xax, yax], animation_duration=1000, interaction=intsel)
display(fig)

Figure(animation_duration=1000, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='y', ori…

In [28]:
len(scatter.selected) if scatter.selected is not None else None

22

In [29]:
# Changing to a vertical brush selector

fig.interaction = bq.interacts.BrushIntervalSelector(scale=ys, marks=[scatter],
                                                     orientation='vertical')

In [ ]:
# Changing to a lasso selector
#
#fig.interaction = bq.interacts.LassoSelector(x_scale=xs, y_scale=ys, color='olive', marks=[scatter])

In [ ]:
fig.interaction.reset()

# Handdraw

Changing the interaction of the previous figure to be a hand draw

In [30]:
lines = bq.Lines(x=x, y=y, scales={'x': xs, 'y': ys})
fig = bq.Figure(marks=[lines], axes=[xax, yax], animation_duration=1000)

handdraw = bq.interacts.HandDraw(lines=lines)
fig.interaction = handdraw

display(fig)

Figure(animation_duration=1000, axes=[Axis(label='x', scale=LinearScale(), side='bottom'), Axis(label='y', ori…

In [31]:
# Observing the line changes

def line_change(change):
    db.value = str(change['new'])

lines.observe(line_change,
              names=['y'] )

db = widgets.Label()
db.value = str(lines.y)
db

Label(value='[-0.52867066 -0.6366745  -1.3775412  -0.65496666 -0.65496666 -0.61853701\n -0.54567772 -0.4728184…

# Moving points around

In [32]:
from bqplot import *

size = 100
np.random.seed(0)
x_data = range(size)
y_data = np.cumsum(np.random.randn(size) * 100.0)

## Enabling moving of points in scatter. Try to click and drag any of the points in the scatter and 
## notice the line representing the mean of the data update

sc_x = LinearScale()
sc_y = LinearScale()

scat = Scatter(x=x_data[:10], y=y_data[:10], scales={'x': sc_x, 'y': sc_y}, default_colors=['blue'],
               enable_move=True)
lin = Lines(scales={'x': sc_x, 'y': sc_y}, stroke_width=4, line_style='dashed', colors=['orange'])
m = Label(value='Mean is %s'%np.mean(scat.y))

def update_line(change):
    with lin.hold_sync():
        lin.x = [np.min(scat.x), np.max(scat.x)]
        lin.y = [np.mean(scat.y), np.mean(scat.y)]
        m.value='Mean is %s'%np.mean(scat.y)
        

update_line(None)

# update line on change of x or y of scatter
scat.observe(update_line, names='x')
scat.observe(update_line, names='y')

ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, tick_format='0.2f', orientation='vertical')

fig = Figure(marks=[scat, lin], axes=[ax_x, ax_y])

## In this case on drag, the line updates as you move the points.
with scat.hold_sync():
    scat.enable_move = True
    scat.update_on_move = True
    scat.enable_add = False

display(m, fig)

Label(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], interactions={'hover': 'tooltip'}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}}, tooltip_style={'opacity': 0.9})

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale(), tick_format='0.2f')]…

# A more complex example

Although [bqplot](https://github.com/bloomberg/bqplot) is a versatile plotting package, one of its unique strengths is that the plot is a widget. Plot elements, like the position of points on the graph, can be monitored for changes like any other widget trait. 

The example in this notebook graphs a Fourier sine series using [bqplot](https://github.com/bloomberg/bqplot) to graph the series and a separate [bqplot](https://github.com/bloomberg/bqplot) to allow users to sest the amplitude of the terms in the series.

## Define the function that calculates the Fourier series

Though in a real application the plot range and number of points should perhaps be configurable via a widget, for this example they are hard coded.

In [33]:
def fourier_series(amplitudes):
    """
    Compute the fourier sine series given a set of amplitudes. The 
    period of the fundamental of the series is 1 and the series is 
    generated for two periods.
    """
    period = 1.0
    x = np.linspace(0, 2 * period, num=1000)
    y = np.sum(a * np.sin(2 * np.pi * (n + 1) * x / period) 
               for n, a in enumerate(amplitudes))

    return x, y

The number of Fourier components in the series should probably also be user-configurable; for the sake of simplicity it is hard coded here. We also define some test data so we can look at plot for setting the amplitudes before we connect it up to the plot of the Fourier series sum.

In [34]:
N_fourier_components = 10
x_data = np.arange(N_fourier_components) + 1
y_data = np.random.uniform(low=-1, high=1, size=N_fourier_components)

## Create the amplitude control

We will create both this and the plot of the series using the "Grammar of Graphics" interface to bqplot. That makes it easier to treat the plot as a widget.

Please read through and execute the example below.

In [35]:
# Start by defining a scale for each axis
sc_x = LinearScale()

# The amplitudes are limited to ±1 for this example...
sc_y = LinearScale(min=-1.0, max=1.0)

# You can create a Scatter object without supplying the data at this
# point. It is here so we can see how the control looks.
scat = Scatter(x=x_data, y=y_data, 
               scales={'x': sc_x, 'y': sc_y}, 
               colors=['orange'],
               # This is what makes this plot interactive
               enable_move=True)

# Only allow points to be moved vertically...
scat.restrict_y = True

# Define the axes themselves
ax_x = Axis(scale=sc_x)
ax_y = Axis(scale=sc_y, tick_format='0.2f', orientation='vertical')

# The graph itself...
amplitude_control = Figure(marks=[scat], axes=[ax_x, ax_y], 
                           title='Fourier amplitudes')

# Let's see what this looks like...
amplitude_control

Figure(axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale(max=1.0, min=-1.0), tic…

Try dragging the points on the graph around. Print the $y$-data from the plot (`scat.y`) and the original data below you they should be different.

## Set up some initial conditions

To test our sine series plot it is helpful to start with a simple-to-understand series: just the fundmental, with amplitude 1. 

In [36]:
# Add some test data to make view the result
initial_amplitudes = np.zeros(10)
initial_amplitudes[0] = 1.0

## Create the plot of the Fourier series

As above, we create the plot using the Grammar of Graphics interface.

In [37]:
lin_x = LinearScale()
lin_y = LinearScale()

# Note that here, unlike above, we do not set the initial data.
line = Lines(scales={'x': lin_x, 'y': lin_y}, colors=['orange'],
               enable_move=False)

ax_x = Axis(scale=lin_x)
ax_y = Axis(scale=lin_y, tick_format='0.2f', orientation='vertical')

result = Figure(marks=[line], axes=[ax_x, ax_y], 
                title='Fourier sine series',
                animation_duration=500)

# Calculate the fourier series....
line.x, line.y = fourier_series(initial_amplitudes)

# Let's take a look!
result

Figure(animation_duration=500, axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale…

### Set the amplitude control to match this initial case

Note that you can access the `scat` object from the `Figure` widget. Each line, scatter or other mark on the plot is in the list at `.marks`.

In [38]:
amplitude_control.marks[0].y = initial_amplitudes

## You make the widget out of `amplitude_control` and `result`

See the animation above for a reminder of the target. Dragging the amplitudes will *not* change the sine series yet.

In [39]:
# %load solutions/bqplot-as-control/box-widget.py
box = widgets.HBox(children=[amplitude_control, result])
box

## Looks good, connect things up

Fill in the body of the function below, which will be observed by `amplitude_control`. You should call `fourier_series` and set the appropriate line properties.

In [40]:
def update_line(change):
    new_x, new_y = fourier_series(change['new'])
    line.x = new_x
    line.y = new_y

In [41]:
# React to changes in the y value....
amplitude_control.marks[0].observe(update_line, names=['y'])